In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from io import StringIO

#Generar un dataset de partidos

In [2]:
def resultados(Gol_local, Gol_visitante):
  Resultado = []
  for i, j in zip(Gol_local, Gol_visitante):
    if i > j:
      Resultado.append('V')
    elif i < j:
      Resultado.append('D')
    else:
      Resultado.append('E')
  return Resultado

In [3]:
temporada = '2023-2024'
Liga = {'La-Liga':12, 'Premier-League':9, 'Serie-A':11, 'Ligue-1':13, 'Bundesliga':20}

In [4]:
def WebScraping(url, Liga):
  data = []
  titulos = ['Sem.','Día','Fecha','Hora','Local','xG_l','Marcador','xG_v','Visitante','Asistencia','Sedes','Árbitro', 'Informe del partido', 'Notas']
  respuesta = requests.get(url)
  if respuesta.status_code == 200:
    s = bs(respuesta.content, 'html.parser')
    tabla = s.find('table')
    filas = tabla.find('tbody').find_all('tr')
    for fila in filas:
      celdas = fila.find_all(['td', 'th'])
      datos_fila = [celda.text.strip() for celda in celdas]
      data.append(datos_fila)
    df = pd.DataFrame(data, columns = titulos)
    df['Liga'] = Liga.replace('-', ' ')
    df = df[df['Sem.'] != '']
    numericos = ['Sem.','xG_l','xG_v','Asistencia']
    df['Asistencia'] = df['Asistencia'].str.replace(',', '')
    df[numericos] = df[numericos].apply(pd.to_numeric)
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M').dt.time
    df[['Gol_local', 'Gol_visitante']] = df.Marcador.str.split('–', expand=True)
    df['resultado'] = resultados(df.Gol_local, df.Gol_visitante)
    df.drop(['Marcador', 'Informe del partido', 'Notas'], axis=1, inplace=True)
    return df
  else:
    print(f'Status {respuesta.status_code}')

In [5]:
df_dict = {}
for i in Liga:
    url = f'https://fbref.com/es/comps/{Liga[i]}/{temporada}/horario/Marcadores-y-partidos-de-{temporada}-{i}'
    df = WebScraping(url, i)
    df_dict[f'df_{i}'] = df

In [6]:
partidos = pd.concat(df_dict.values(), ignore_index=True)

In [7]:
partidos.head(15)

,Sem.,Día,Fecha,Hora,Local,xG_l,xG_v,Visitante,Asistencia,Sedes,Árbitro,Liga,Gol_local,Gol_visitante,resultado
0,1,Vie,2023-08-11,19:30:00,Almería,1.4,2.1,Rayo Vallecano,14837.0,Power Horse Stadium,Javier Alberola,La Liga,0,2,D
1,1,Vie,2023-08-11,22:00:00,Sevilla,0.7,1.1,Valencia,34184.0,Estadio Ramón Sánchez Pizjuán,José Sánchez,La Liga,1,2,D
2,1,Sáb,2023-08-12,17:00:00,Real Sociedad,1.0,0.8,Girona,30848.0,Reale Arena,Francisco Hernández,La Liga,1,1,E
3,1,Sáb,2023-08-12,18:30:00,Las Palmas,0.9,1.6,Mallorca,24100.0,Estadio de Gran Canaria,Víctor García,La Liga,1,1,E
4,1,Sáb,2023-08-12,21:30:00,Athletic Club,0.4,0.9,Real Madrid,48927.0,San Mamés,Jesús Gil,La Liga,0,2,D
5,1,Dom,2023-08-13,17:00:00,Celta Vigo,1.0,1.0,Osasuna,20269.0,Estadio Abanca Balaídos,Pablo González,La Liga,0,2,D
6,1,Dom,2023-08-13,19:30:00,Villarreal,0.9,1.1,Betis,17825.0,Estadio de la Cerámica,Miguel Ángel Ortiz Arias,La Liga,1,2,D
7,1,Dom,2023-08-13,21:30:00,Getafe,0.6,1.4,Barcelona,13410.0,Coliseum Alfonso Pérez,César Soto,La Liga,0,0,E
8,1,Lun,2023-08-14,19:30:00,Cádiz,0.7,0.3,Alavés,19306.0,Estadio Nuevo Mirandilla,Isidro Díaz de Mera,La Liga,1,0,V
9,1,Lun,2023-08-14,21:30:00,Atlético Madrid,2.4,0.9,Granada,56164.0,Estadio Cívitas Metropolitano,Juan Pulido,La Liga,3,1,V


In [8]:
ruta = '../Data/partidos.csv'

In [9]:
partidos.to_csv(ruta, index=False)

#Generar in dataset de puntuacion Elo de acuerdo a [Elo.com](http://clubelo.com/API)

In [10]:
clubes_dict = {
    'Almeria': 'Almería',
    'Sevilla': 'Sevilla',
    'Sociedad': 'Real Sociedad',
    'Las Palmas': 'Las Palmas',
    'Bilbao': 'Athletic Club',
    'Celta': 'Celta Vigo',
    'Villarreal': 'Villarreal',
    'Getafe': 'Getafe',
    'Cadiz': 'Cádiz',
    'Atletico': 'Atlético Madrid',
    'Mallorca': 'Mallorca',
    'Valencia': 'Valencia',
    'Osasuna': 'Osasuna',
    'Girona': 'Girona',
    'Barcelona': 'Barcelona',
    'Betis': 'Betis',
    'Alaves': 'Alavés',
    'Granada': 'Granada',
    'Rayo Vallecano': 'Rayo Vallecano',
    'Real Madrid': 'Real Madrid',
    'Burnley': 'Burnley',
    'Arsenal': 'Arsenal',
    'Everton': 'Everton',
    'Sheffield United': 'Sheffield Utd',
    'Brighton': 'Brighton',
    'Bournemouth': 'Bournemouth',
    'Newcastle': 'Newcastle Utd',
    'Brentford': 'Brentford',
    'Chelsea': 'Chelsea',
    'Man United': 'Manchester Utd',
    'Forest': "Nott'ham Forest",
    'Fulham': 'Fulham',
    'Liverpool': 'Liverpool',
    'Wolves': 'Wolves',
    'Tottenham': 'Tottenham',
    'Man City': 'Manchester City',
    'Aston Villa': 'Aston Villa',
    'West Ham': 'West Ham',
    'Crystal Palace': 'Crystal Palace',
    'Luton': 'Luton Town',
    'Frosinone': 'Frosinone',
    'Empoli': 'Empoli',
    'Inter': 'Inter',
    'Genoa': 'Genoa',
    'Sassuolo': 'Sassuolo',
    'Roma': 'Roma',
    'Lecce': 'Lecce',
    'Udinese': 'Udinese',
    'Torino': 'Torino',
    'Bologna': 'Bologna',
    'Monza': 'Monza',
    'Verona': 'Hellas Verona',
    'Milan': 'Milan',
    'Juventus': 'Juventus',
    'Fiorentina': 'Fiorentina',
    'Napoli': 'Napoli',
    'Lazio': 'Lazio',
    'Salernitana': 'Salernitana',
    'Cagliari': 'Cagliari',
    'Atalanta': 'Atalanta',
    'Nice': 'Nice',
    'Marseille': 'Marseille',
    'Paris SG': 'Paris S-G',
    'Brest': 'Brest',
    'Nantes': 'Nantes',
    'Clermont': 'Clermont Foot',
    'Montpellier': 'Montpellier',
    'Rennes': 'Rennes',
    'Strasbourg': 'Strasbourg',
    'Metz': 'Metz',
    'Lyon': 'Lyon',
    'Toulouse': 'Toulouse',
    'Lille': 'Lille',
    'Reims': 'Reims',
    'Le Havre': 'Le Havre',
    'Lorient': 'Lorient',
    'Monaco': 'Monaco',
    'Lens': 'Lens',
    'Werder': 'Werder Bremen',
    'Leverkusen': 'Leverkusen',
    'Wolfsburg': 'Wolfsburg',
    'Stuttgart': 'Stuttgart',
    'Augsburg': 'Augsburg',
    'Hoffenheim': 'Hoffenheim',
    'Dortmund': 'Dortmund',
    'Union Berlin': 'Union Berlin',
    'Frankfurt': 'Eint Frankfurt',
    'RB Leipzig': 'RB Leipzig',
    'Bochum': 'Bochum',
    'Darmstadt': 'Darmstadt 98',
    'Freiburg': 'Freiburg',
    'Koeln': 'Köln',
    'Heidenheim': 'Heidenheim',
    'Gladbach': 'Gladbach',
    'Mainz': 'Mainz 05',
    'Bayern': 'Bayern Munich'
}

In [11]:
def WebScrapingElo(url, ligas, año):
  respuesta = requests.get(url)
  if respuesta.status_code == 200:
    data = StringIO(respuesta.text)
    df = pd.read_csv(data, index_col = 'Rank')
    df = df[df['Country'].isin(ligas) & df['Level'] == 1]
    df['Fecha'] = año
    df.replace(clubes_dict, inplace=True)
    df.drop(['From', 'To', 'Level', 'Country'], axis=1, inplace=True)
    return df
  else:
    print(f'Status {respuesta.status_code}')

In [12]:
años = partidos.Fecha.unique().strftime('%Y-%m-%d')
Ligas = ['ENG', 'GER', 'ESP', 'ITA', 'FRA']

In [13]:
df_dict2 = {}
for i in años:
    url = f'http://api.clubelo.com/{i}'
    df = WebScrapingElo(url, Ligas, i)
    df_dict2[f'df_{i}'] = df

In [14]:
Puntuacion = pd.concat(df_dict2.values(), ignore_index=True)

In [15]:
Puntuacion

,Club,Elo,Fecha
0,Manchester City,2077.252197,2023-08-11
1,Liverpool,1946.900024,2023-08-11
2,Bayern Munich,1935.604248,2023-08-11
3,Arsenal,1919.369019,2023-08-11
4,Real Madrid,1906.480347,2023-08-11
...,...,...,...
17755,Lorient,1553.590698,2024-02-07
17756,Nantes,1544.605835,2024-02-07
17757,Almería,1536.975830,2024-02-07
17758,Metz,1532.317505,2024-02-07


In [16]:
ruta2 = '../Data/puntuacion.csv'

In [17]:
Puntuacion.to_csv(ruta2, index=False)